In [12]:
import sys
import importlib
import subprocess

def install_if_colab():
    if "google.colab" in sys.modules:
        print("Running in Google Colab. Checking required libraries...")

        packages = ["moabb", "braindecode"]  # Add required libraries
        missing_packages = [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]

        if missing_packages:
            print(f"Installing missing libraries: {', '.join(missing_packages)}")
            !pip install {" ".join(missing_packages)}
        else:
            print("All required libraries are already installed.")
    else:
        print("Not running in Google Colab. No installation needed.")

install_if_colab()


Running in Google Colab. Checking required libraries...
All required libraries are already installed.


In [13]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
with open(path+'EEG_data/test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)

import pickle
with open(path+'EEG_data/train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)

In [ ]:
import torch
#from shallow_fbcsp import ShallowFBCSPNet
from braindecode.util import set_random_seeds


cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 19123
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_channels = 22
input_window_samples = 1125

print("n_classes: ", n_classes)
print("n_channels:", n_channels)
print("input_window_samples size:", input_window_samples)

In [ ]:
#from models_fbscp import CollapsedShallowNet
# The ShallowFBCSPNet is a `nn.Sequential` model
import importlib
from weight_init import init_weights
import LSTM_model
importlib.reload(LSTM_model)
from LSTM_model import ShallowLSTMNet
model = ShallowLSTMNet(
    n_chans=22,
    n_outputs=n_classes,
    n_times=input_window_samples,
)

# Display torchinfo table describing the model
print(model)

# Send model to GPU
if cuda:
    model.cuda()



In [ ]:
# splitted = windows_dataset.split("session")
# train_set = splitted['0train']  # Session train
# test_set = splitted['1test']  # Session evaluation

from torch.nn import Module
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader

#lr = 1e-4
#weight_decay = 1e-4
#batch_size = 64
#n_epochs = 200


In [ ]:

from tqdm import tqdm
# Define a method for training one epoch


def train_one_epoch(
        dataloader: DataLoader, model: Module, loss_fn, optimizer,
        scheduler: LRScheduler, epoch: int, device, print_batch_stats=True
):
    model.train()  # Set the model to training mode
    train_loss, correct = 0, 0

    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader),
                        disable=not print_batch_stats)

    for batch_idx, (X, y, _) in progress_bar:
        X, y = X.to(device), y.to(device)
        #print(X.shape)
        optimizer.zero_grad()
        pred = model(X)
        #print(y.shape)
        #print(pred.shape)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()  # update the model weights
        optimizer.zero_grad()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).sum().item()

        #if print_batch_stats:
        #    progress_bar.set_description(
        #        f"Epoch {epoch}/{n_epochs}, "
        #        f"Batch {batch_idx + 1}/{len(dataloader)}, "
        #        f"Loss: {loss.item():.6f}"
        #    )

    # Update the learning rate
    scheduler.step()

    correct /= len(dataloader.dataset)
    return train_loss / len(dataloader), correct


In [ ]:
from collections import defaultdict
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np

@torch.no_grad()
def test_model(dataloader: DataLoader, model: torch.nn.Module, loss_fn, print_batch_stats=True):
    device = next(model.parameters()).device  # Get model device
    size = len(dataloader.dataset)
    n_batches = len(dataloader)
    model.eval()  # Switch to evaluation mode
    test_loss, correct = 0, 0

    # Initialize dictionaries for per-class tracking
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    # Lists to store true and predicted labels for confusion matrix
    all_preds = []
    all_targets = []

    if print_batch_stats:
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    else:
        progress_bar = enumerate(dataloader)

    for batch_idx, (X, y, _) in progress_bar:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        batch_loss = loss_fn(pred, y).item()

        test_loss += batch_loss
        correct += (pred.argmax(1) == y).sum().item()

        # Store predictions and true labels for confusion matrix
        all_preds.append(pred.argmax(1).cpu())
        all_targets.append(y.cpu())

        # Compute per-class accuracy
        preds_labels = pred.argmax(1)
        for label, pred_label in zip(y, preds_labels):
            class_total[label.item()] += 1
            class_correct[label.item()] += (label == pred_label).item()

        if print_batch_stats:
            progress_bar.set_description(
                f"Batch {batch_idx + 1}/{len(dataloader)}, Loss: {batch_loss:.6f}"
            )

    # Convert lists to tensors
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)

    # Compute per-class accuracy
    class_accuracies = {
        cls: (class_correct[cls] / class_total[cls]) * 100 if class_total[cls] > 0 else 0
        for cls in class_total
    }

    # Compute overall accuracy
    test_loss /= n_batches
    overall_accuracy = (correct / size) * 100

    # Print per-class accuracy
    print("\nClass-wise Accuracy:")
    for cls, acc in class_accuracies.items():
        print(f"  Class {cls}: {acc:.2f}%")

    print(f"Test Accuracy: {overall_accuracy:.1f}%, Test Loss: {test_loss:.6f}\n")

    return test_loss, overall_accuracy, class_accuracies, all_preds, all_targets


In [ ]:
import wandb
wandb.login()

In [ ]:
import torch
import wandb
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import CrossEntropyLoss
import numpy as np
import math

seeds = [99999,182726,91111222,44552222,12223111,100300,47456655,4788347,77766666,809890]
for _seed in seeds:
  seed = _seed
  set_random_seeds(seed=seed, cuda=cuda)

  model = ShallowLSTMNet(
      n_chans=22,
      n_outputs=n_classes,
      n_times=input_window_samples,
  )

  # Display torchinfo table describing the model
  print(model)

  # Send model to GPU
  if cuda:
      model.cuda()

  wandb.init(project="Master Thesis", name=f"LSTM {seed}")
  model.apply(init_weights)

  lr = 1e-3
  weight_decay = 1e-4
  batch_size = 64
  n_epochs = 100

  final_acc = 0.0

  patience = 10  # Early stopping patience
  best_val_loss = float("inf")
  counter = 0  # Counts epochs without improvement

  wandb.config.update({
      "learning_rate": lr,
      "weight_decay": weight_decay,
      "batch_size": batch_size,
      "epochs": n_epochs
  })



  # Define optimizer and scheduler
  optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs - 1)

  # Define loss function
  loss_fn = CrossEntropyLoss()

  # Create DataLoaders
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_set, batch_size=batch_size)

  # Initialize lists to store all predictions & targets
  all_preds, all_targets = [], []
  #wandb.watch(model, log="all")
  # Training loop
  for epoch in range(1, n_epochs + 1):
      print(f"Epoch {epoch}/{n_epochs}: ", end="")

      train_loss, train_accuracy = train_one_epoch(
          train_loader, model, loss_fn, optimizer, scheduler, epoch, device
      )

      test_loss, test_accuracy, class_accuracies, batch_preds, batch_targets = test_model(test_loader, model, loss_fn)
      final_acc = test_accuracy
      # Store predictions & labels for confusion matrix
      all_preds.extend(batch_preds)
      all_targets.extend(batch_targets)

      # Print class-wise accuracy
      print("\nClass-wise Accuracy:")
      for class_idx, acc in class_accuracies.items():
          print(f"  Class {class_idx}: {acc:.2f}%")

      # Log results to wandb
      wandb.log({
          "epoch": epoch,
          "train_loss": train_loss,
          "train_accuracy": train_accuracy * 100,
          "test_loss": test_loss,
          "test_accuracy": test_accuracy,
          "learning_rate": scheduler.get_last_lr()[0],
          **{f"class_{class_idx}_accuracy": acc for class_idx, acc in class_accuracies.items()}
      })

      print(
          f"Train Accuracy: {100 * train_accuracy:.2f}%, "
          f"Average Train Loss: {train_loss:.6f}, "
          f"Test Accuracy: {test_accuracy:.2f}%, "
          f"Average Test Loss: {test_loss:.6f}\n"
      )
      # Early Stopping Logic
      if test_loss < best_val_loss:
          best_val_loss = test_loss
          counter = 0  # Reset counter if validation loss improves
          torch.save(model.state_dict(), "best_model.pth")  # Save best model
      else:
          counter += 1  # Increment if no improvement

      if counter >= patience:
          print(f"Early stopping triggered after {patience} epochs of no improvement.")
          break  # Stop training


  # Convert lists to NumPy arrays
  all_preds = np.array(all_preds)
  all_targets = np.array(all_targets)

  # Save predictions & true labels for later use (confusion matrix)
  wandb.log({"all_preds": all_preds.tolist(), "all_targets": all_targets.tolist()})
  wandb.finish()

  # Assuming 'model' is your trained Braindecode model
  torch.save(model, f"LSTM_{math.ceil(final_acc)}_{seed}.pth")
  torch.save(model.state_dict(), f"LSTM_{math.ceil(final_acc)}_{seed}_state.pth")